# 1 Processing

In [9]:
import os
HOME = os.getcwd()
print(HOME)

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO


In [3]:
#clearml mac
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=QO1VBBX9J2S2VYILQTGI
# %env CLEARML_API_SECRET_KEY=ERuc1S6o5SirQGugvYXDFjH9b9aNi0u8S3rpALzXMa8YPSLDMW

# clearml win
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
%env CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
env: CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn


In [4]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.41 🚀 Python-3.12.2 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
Setup complete ✅ (24 CPUs, 125.5 GB RAM, 319.0/459.6 GB disk)


/home/jpan/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [14]:
# read config file
import yaml

def load_config(config_file):
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    return config

config = load_config('seg_yolo_config_4.yaml')

print(config)

# project
project = config['project']
task_name = config['task_name']
exist_ok = config['exist_ok']

# dataset
# data = config['data_mac']
data = config['data_win']
test_data = config['test_data']
imgsz = config['imgsz']

# model
model = config['model']
task = config['task']
mode = config['mode']
# device = config['device_mac']
device = config['device_win']

# Training parameters
epochs = config['epochs']
batch = config['batch']
lr0 = config['lr0']
cos_lr = config['cos_lr']
weight_decay = config['weight_decay']
dropout = config['dropout']
optimizer = config['optimizer']
momentum = config['momentum']
seed = config['seed']

# Output configuration
# save = config['save']
# save_period = config['save_period']
plots = config['plots']
test_output = config['test_output']


{'project': 'YOLO-v8-seg', 'task_name': 'YOLO-seg-test-4', 'exist_ok': False, 'data_mac': '/Users/holmes/Documents/UNI-Bamberg/Arbeiten/Datensatz/Selfmade/3_iphone_domini_seg/YOLO/Dominik-seg.v2_yolo.yolov8/data.yaml', 'data_win': '/mnt/data/backup_pan/Dataset/Seg-dominik-iphone/YOLO/Dominik-seg.v2_yolo.yolov8/data.yaml', 'test_data': '/path/to/test_data.txt', 'imgsz': 640, 'model': 'yolov8m-seg.pt', 'task': 'segment', 'mode': 'train', 'device_mac': 'mps', 'device_win': 0, 'epochs': 200, 'batch': 16, 'lr0': 0.001, 'cos_lr': True, 'weight_decay': 0, 'dropout': 0.0, 'optimizer': 'Adam', 'momentum': 0.9, 'seed': 42, 'save': False, 'save_period': -1, 'plots': True, 'test_output': '/path/to/test_output'}


In [15]:
import pandas as pd
import math

Dataset_version = "v2"
train_amount = 1113
batch_size = batch
total_epoch =  epochs
base_lr = lr0

test_record_csv = "/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/Test_seg_yolo_record.csv"

# Save in csv
new_row = [
    task_name,
    Dataset_version,
    model,
    train_amount,
    batch_size,
    total_epoch,
    base_lr,
    "none"
]

df = pd.read_csv(test_record_csv, header=None)
df.loc[len(df)] = new_row

df.to_csv(test_record_csv, header=False, index=False)

In [16]:
import clearml
from clearml import Task

#Clear ML Initialization
cl_task = Task.init(project_name=project,task_name=task_name)
logger = cl_task.get_logger()

ClearML Task: created new task id=a7a2824309c64792bc967af9788ded54
ClearML results page: https://app.clear.ml/projects/2416121eefb740c29276a6cba32fd03b/experiments/a7a2824309c64792bc967af9788ded54/output/log


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


# 2 Training

In [17]:
from ultralytics import YOLO

ini_model = "/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/yolov8m-seg.pt"
model = YOLO(ini_model)

results = model.train(project=project, name=task_name, exist_ok=exist_ok, data=data, imgsz=imgsz, task=task, device=device, epochs=epochs, batch=batch, lr0=lr0, cos_lr=cos_lr, weight_decay=weight_decay, dropout=dropout, optimizer=optimizer, momentum=momentum, seed=seed, plots=plots)

cl_task.close()

New https://pypi.org/project/ultralytics/8.2.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.41 🚀 Python-3.12.2 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
engine/trainer: task=segment, mode=train, model=/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/yolov8m-seg.pt, data=/mnt/data/backup_pan/Dataset/Seg-dominik-iphone/YOLO/Dominik-seg.v2_yolo.yolov8/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=YOLO-v8-seg, name=YOLO-seg-test-4, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_str

train: Scanning /mnt/data/backup_pan/Dataset/Seg-dominik-iphone/YOLO/Dominik-seg.v2_yolo.yolov8/train/labels.cache... 1119 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1119/1119 [00:00<?, ?it/s]
val: Scanning /mnt/data/backup_pan/Dataset/Seg-dominik-iphone/YOLO/Dominik-seg.v2_yolo.yolov8/valid/labels.cache... 107 images, 0 backgrounds, 0 corrupt: 100%|██████████| 107/107 [00:00<?, ?it/s]


Plotting labels to YOLO-v8-seg/YOLO-seg-test-4/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.9) with parameter groups 86 weight(decay=0.0), 97 weight(decay=0.0), 96 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to YOLO-v8-seg/YOLO-seg-test-4
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/200      8.07G       1.18      2.747      1.853      1.444        143        640: 100%|██████████| 70/70 [00:18<00:00,  3.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.20it/s]

                   all        107        393      0.489      0.624      0.509      0.353      0.409      0.464      0.367      0.209


...

                   all        107        393      0.682      0.744      0.721      0.641      0.706      0.705      0.704      0.589
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 54, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



154 epochs completed in 0.989 hours.
Optimizer stripped from YOLO-v8-seg/YOLO-seg-test-4/weights/last.pt, 54.8MB
Optimizer stripped from YOLO-v8-seg/YOLO-seg-test-4/weights/best.pt, 54.8MB

Validating YOLO-v8-seg/YOLO-seg-test-4/weights/best.pt...
Ultralytics YOLOv8.2.41 🚀 Python-3.12.2 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
YOLOv8m-seg summary (fused): 245 layers, 27223542 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.45it/s]


                   all        107        393      0.726      0.784       0.78      0.681       0.72      0.778      0.769      0.628
             main_beam         71         75      0.786       0.88       0.89      0.825      0.786       0.88      0.885      0.822
             side_beam         82        318      0.667      0.689       0.67      0.537      0.654      0.677      0.653      0.434
Speed: 0.4ms preprocess, 4.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to YOLO-v8-seg/YOLO-seg-test-4


Exception encountered while uploading Failed uploading object /YOLO-v8-seg/YOLO-seg-test-4.a7a2824309c64792bc967af9788ded54/models/best.pt (413): <!doctype html>
<html lang=en>
<title>413 Request Entity Too Large</title>
<h1>Request Entity Too Large</h1>
<p>The data value transmitted exceeds the capacity limit.</p>



# 3 testing

## 3.1 path

In [ ]:
from ultralytics import YOLO
import cv2
import os

model_path = "/Users/holmes/Documents/UNI-Bamberg/Arbeiten/Uni_Bamberg_KDWT_Holzprojekt/Project_process/P2_Experiments/202403_Schweinfurt/Uni-Wood/Schweinfurt-Yolo-Test-4/weights/best.pt"

image_folder = "/Users/holmes/Desktop/probe"

save_path = "/Users/holmes/Desktop/probe"



## 3.2 resize

In [ ]:
# Resize images
resized_image_folder = os.path.join(image_folder, "resized")
os.makedirs(resized_image_folder, exist_ok=True)

for file in os.listdir(image_folder):
    if file.endswith(('.jpg', '.JPG', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (640, 640))
        resized_img_path = os.path.join(resized_image_folder, os.path.splitext(file)[0] + "_resize" + os.path.splitext(file)[1])
        cv2.imwrite(resized_img_path, img_resized)
image_folder = resized_image_folder


## 3.3 test

In [ ]:
model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    results = model(img_path)
    img = cv2.imread(img_path)

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            # box contains prediction information
            confidence = box.conf[0]
            if confidence > 0.4:
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 2)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    unique_save_path = os.path.join(save_path, os.path.basename(img_path))
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")
